In [1]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv("./Documents/COVID_for_families.csv")
df_backup = pd.read_csv("./Documents/COVID_for_families.csv")

In [31]:
df.shape

(7305, 119)

## FASI DA AFFRONTARE
1. Pulizia del dataset
    - eliminare le features non utili
    - eliminare le righe contenenti troppi campi nulli
2. Features extraction

3. Clustering
4. Visualizzazione clusters
5. Analisi di associazione di pattern interessanti
6. Conclusioni

In [32]:
for name in df.columns:
    print(name)

ID
TemporaryMark
Gender
AutonomousCommunity
SchoolOwnership
Pond
PondMax3
KindOfPlace
Income
LivingUnit
DependentPersons
WorkConfinement
WorkConfinementsSecondAdult
ChildEarlyEducation1
ChildEarlyEducation2
ChildPrimaryEducation
ChildSecondaryEducation
ChildBaccaleaurate
ChildVocationalTraining
ChildrenSpecialNeeds
WorkloadChange
HouseworkMore
Sleep
Leisure
Sports
Friends
ExtendedFamily
Past
Future
Sex
Reconciling
DomesticHelp
Tired
TimeOff
InterruptChildren
InterruptChildrenFrequency
ChildrenSchookwork
ShareElectronicDevices
Bedtime
PositiveImpact
Bonding
BondingNeighbours
TastesAndAbilities
Responsability
Adaptarion
Understanding
NegativeImpact
Anxiety
Loneliness
Sadness
LossFamilyMember
FinancialLoss
Arguments
AlcoholMedicationDrug
TechnologyUse
NegativeImpactOther
NegativeImpactNoanswer
ChildPhysicalActivity
ChildScreens
ChildSleep
ChildDiet
ChildFear
ChildFrustation
Irritability
Boredom
ChildFake
ChildIFriends
ChildFriendship
ChildSpace
ChildIntrospection
ChildLeavingHome
ChildNoV

## 1. Data preliminar analisys
- Explore the dataset to detect missing values, outliers, and other necessary transformations
- Describe the distribution of the main socio-demographic variables

#### Note:
Dataset unbalanced for the gendere 2 (female)

In [33]:
df.Gender.value_counts()

Gender
2.0    6686
1.0     600
Name: count, dtype: int64

#### Note:
the First Provinces where the responents (76%) lives are:
- COMUNIDAD VALENCIANA
- MADRID
- CATALUÑA
- ANDALUCÍA

In [34]:
df.AutonomousCommunity.value_counts()

AutonomousCommunity
COMUNIDAD VALENCIANA    3125
MADRID                  1173
CATALUÑA                 715
ANDALUCÍA                535
PAIS VASCO               235
CASTILLA Y LEÓN          234
CASTILLA LA MANCHA       223
GALICIA                  212
ARAGÓN                   159
CANARIAS                 150
ASTURIAS                 124
MURCIA                   104
ILLES BALEARS             86
CANTABRIA                 67
NAVARRA                   64
EXTREMADURA               57
LA RIOJA                  31
CEUTA                      6
MELILLA                    4
Name: count, dtype: int64

#### Note:
Most of them live in Big city & send their kids to public school

In [35]:
df.KindOfPlace.value_counts()

KindOfPlace
2.0    4273
1.0    1805
0.0    1027
Name: count, dtype: int64

In [36]:
df.SchoolOwnership.value_counts()

SchoolOwnership
Público       4558
Concertado    1810
Privado        937
Name: count, dtype: int64

#### Note:
Most of the family are composed with "Two adults with a minor or minors in their care"

In [37]:
df.LivingUnit.value_counts()

LivingUnit
2    6262
1     699
3     344
Name: count, dtype: int64

#### Preliminar filtering
Before any processing, I drop some columns which either have too many NaN values or are useless.

Then I drop all the istances that have at least 2 NaN values (excluding 'Income' column).

I don't remove all columns with at least one NaN value because they are important columns for the analysis and the percentage of NaN values ​​for each column is very low (1% over all instances).
Same, I don't drop all instances with at least one NaN values because there are 5766 instances that match this status...

In [38]:
drop_column_list = ['ID',
                    'TemporaryMark', 
                    'Pond',
                    'PondMax3', 
                    'ShareElectronicDevices', 
                    'NegativeImpactNoanswer', 
                    'SchoolFees', 
                    'Selección', 
                    'filter_', 
                    'SchoolOwnership1'
                    ]

df.drop(drop_column_list, axis=1, inplace=True)

In [39]:
df.head()

,Gender,AutonomousCommunity,SchoolOwnership,KindOfPlace,Income,LivingUnit,DependentPersons,WorkConfinement,WorkConfinementsSecondAdult,ChildEarlyEducation1,...,SpaceGym,SpaceOther,SpaceNone,SpaceStreet,SpacePlots,SpaceParks,SpacePublic,SpaceSurroundingOther,SpaceSurroundingNone,ActivitiesOutside
0,NaN,PAIS VASCO,Público,2.0,28443.0,2,0,1,1.0,1,...,0,0,0,0,0,0,1,0,0,2.0
1,2.0,PAIS VASCO,Público,2.0,29505.0,2,0,1,0.0,1,...,0,0,0,1,0,1,1,0,0,2.0
2,2.0,PAIS VASCO,Público,2.0,29505.0,2,0,1,1.0,0,...,0,0,0,0,0,1,1,0,0,2.0
3,2.0,PAIS VASCO,Concertado,2.0,29505.0,2,0,1,1.0,1,...,0,0,0,1,0,1,0,0,0,3.0
4,2.0,PAIS VASCO,Público,2.0,29505.0,2,0,1,1.0,0,...,0,1,0,0,0,1,0,0,0,3.0


In [40]:
# Columns with a least one nan value
df.columns[df.isna().any()]

Index(['Gender', 'AutonomousCommunity', 'KindOfPlace', 'Income',
       'WorkConfinementsSecondAdult', 'WorkloadChange', 'HouseworkMore',
       'Sleep', 'Leisure', 'Sports', 'Friends', 'ExtendedFamily', 'Past',
       'Future', 'Sex', 'Reconciling', 'DomesticHelp', 'TimeOff',
       'InterruptChildren', 'InterruptChildrenFrequency', 'ChildrenSchookwork',
       'Bedtime', 'PositiveImpact', 'NegativeImpact', 'HelpOnline',
       'SchoolCommunication', 'ParentsGroups', 'ParentsCommunicationReturn',
       'FamiliesCollaboration', 'SchoolSpace', 'ActivitiesOutside'],
      dtype='object')

In [48]:
# Columns with a least one nan value
na_columns = df.columns[df.isna().any()]

na_columns = na_columns.drop("Income")

# rows with a least 2 nan value
na_rows = df[na_columns].isna().sum(axis=1) >= 2

df_lessToManyNa = df.loc[na_rows, na_columns]
df_lessToManyNa


,Gender,AutonomousCommunity,KindOfPlace,WorkloadChange,HouseworkMore,Sleep,Leisure,Sports,Friends,ExtendedFamily,...,Bedtime,PositiveImpact,NegativeImpact,HelpOnline,SchoolCommunication,ParentsGroups,ParentsCommunicationReturn,FamiliesCollaboration,SchoolSpace,ActivitiesOutside
0,NaN,PAIS VASCO,2.0,3.0,2.0,2.0,2.0,2.0,3.0,3.0,...,1.0,2.0,2.0,NaN,3.0,3.0,1.0,3.0,1.0,2.0
1,2.0,PAIS VASCO,2.0,4.0,2.0,3.0,3.0,1.0,1.0,4.0,...,0.0,1.0,2.0,3.0,4.0,4.0,2.0,2.0,1.0,2.0
3,2.0,PAIS VASCO,2.0,4.0,2.0,2.0,4.0,3.0,2.0,2.0,...,1.0,2.0,2.0,3.0,2.0,3.0,2.0,3.0,1.0,3.0
5,2.0,PAIS VASCO,2.0,6.0,2.0,3.0,1.0,1.0,3.0,1.0,...,0.0,2.0,2.0,3.0,1.0,4.0,3.0,3.0,1.0,3.0
11,2.0,PAIS VASCO,2.0,NaN,2.0,4.0,4.0,3.0,3.0,3.0,...,0.0,2.0,0.0,4.0,3.0,3.0,2.0,3.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7299,2.0,CEUTA,2.0,7.0,0.0,3.0,4.0,3.0,2.0,3.0,...,0.0,2.0,2.0,4.0,2.0,3.0,2.0,3.0,1.0,3.0
7300,1.0,CEUTA,2.0,2.0,2.0,2.0,3.0,3.0,2.0,2.0,...,0.0,1.0,2.0,2.0,NaN,NaN,NaN,3.0,3.0,3.0
7302,2.0,MELILLA,NaN,3.0,1.0,2.0,2.0,1.0,2.0,3.0,...,1.0,0.0,2.0,3.0,2.0,3.0,2.0,3.0,1.0,2.0
7303,2.0,MELILLA,NaN,4.0,2.0,3.0,2.0,2.0,2.0,2.0,...,1.0,2.0,2.0,3.0,2.0,2.0,2.0,3.0,1.0,3.0


In [1056]:
index = [i for i in np.array(df.index) if i not in df_lessToManyNa.index]

df = df.loc[index, :]

In [1057]:
df.shape

(6447, 109)

---

### nan values of '**AutonomousCommunity**' column

- It's just one, I replace its value with mode because it has all the non-NaN features, so it's very significant for the analysis

In [1058]:
df.loc[df.AutonomousCommunity.isna(), :].isna()

,Gender,AutonomousCommunity,SchoolOwnership,KindOfPlace,Income,LivingUnit,DependentPersons,WorkConfinement,WorkConfinementsSecondAdult,ChildEarlyEducation1,...,SpaceGym,SpaceOther,SpaceNone,SpaceStreet,SpacePlots,SpaceParks,SpacePublic,SpaceSurroundingOther,SpaceSurroundingNone,ActivitiesOutside
733,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [1059]:
df.loc[:, ["AutonomousCommunity"]].value_counts()

AutonomousCommunity 
COMUNIDAD VALENCIANA    2790
MADRID                  1062
CATALUÑA                 627
ANDALUCÍA                464
CASTILLA Y LEÓN          208
PAIS VASCO               195
GALICIA                  181
CASTILLA LA MANCHA       178
ARAGÓN                   144
CANARIAS                 125
ASTURIAS                 117
MURCIA                    93
ILLES BALEARS             75
CANTABRIA                 56
NAVARRA                   54
EXTREMADURA               42
LA RIOJA                  27
CEUTA                      4
MELILLA                    4
Name: count, dtype: int64

In [1060]:
AutoCom_mode = df.AutonomousCommunity.mode()[0]
df.loc[df.AutonomousCommunity.isna(), ['AutonomousCommunity']] = AutoCom_mode

---

### nan values of '**KindOfPlace**' column

In [1061]:
df = df.loc[df.KindOfPlace.isna() == False, :]

---

### nan values of '**WorkConfinementsSecondAdult**' column

- If '**LivingUnit**' value is 1 ("**One adult person with a minor or minors in care**") then '**WorkConfinementsSecondAdult**' value is set to 0 ("**No**")

The remaining instances are dropped

In [1062]:
mask_index = df.LivingUnit == 1 & df.WorkConfinementsSecondAdult.isna()

df.loc[mask_index, 'WorkConfinementsSecondAdult'] = 0
# .replace(to_replace=np.nan,value= 0, inplace=True)


In [1063]:
df.loc[df.WorkConfinementsSecondAdult.isna(), ['LivingUnit', 'WorkConfinementsSecondAdult']]

,LivingUnit,WorkConfinementsSecondAdult
319,2,NaN
754,2,NaN
1687,2,NaN
1778,2,NaN
1942,2,NaN
2264,2,NaN
2287,2,NaN
3608,2,NaN
3631,2,NaN
3834,2,NaN


In [1064]:
df = df.loc[df.WorkConfinementsSecondAdult.isna() == False, :]

---

### nan values of '**WorkloadChange**' column

In [1065]:
df = df.loc[df.WorkloadChange.isna() == False, :]

---

### nan values of '**HouseworkMore**' column

In [1066]:
df = df.loc[df.HouseworkMore.isna() == False, :]

---

### nan values of '**Sleep**' column
- If '**Tired**' value is 1 OR 2 ("**Maybe**" OR "**Yes**") then '**Sleep**' value is set to 2 ("**Less than before**")

In [1067]:
Sleep_mode = df.Sleep.mode()[0]
mask_index = df.Sleep.isna() & ((df.Tired == 1) | (df.Tired == 2))

df.loc[mask_index, ['Sleep']] = Sleep_mode

The remaining istances are dropped

In [1068]:
df = df.loc[df.Sleep.isna() == False, :]

---

### nan values of '**DomesticHelp**' column
- Replace all the missing values with meddle values such as "**I prefer not to answer**"

In [1069]:
na_val_count = df.DomesticHelp.isna().sum()
print(f"There are {na_val_count} NaN values")

There are 3 NaN values


In [1070]:
df.loc[df.DomesticHelp.isna(), 'DomesticHelp'] = 2

---

### nan values of '**FamiliesCollaboration**' column
- Replace all the missing values with meddle values such as "**I don't know**"

In [1071]:
na_val_count = df.FamiliesCollaboration.isna().sum()
print(f"There are {na_val_count} NaN values")

There are 13 NaN values


In [1072]:
df.loc[df.FamiliesCollaboration.isna(), 'FamiliesCollaboration'] = 2

---

### nan values of '**PositiveImpact**' column
- The pattern of the next questions is "If yes, then answere....", so if at least one of the next questions values is 1 (**yes**)  then  '**PositiveImpact**' is set to 2 ("**yes**")

In [1073]:
mask_index = df.PositiveImpact.isna() & ((df.Adaptarion==1) | (df.Bonding==1) | (df.TastesAndAbilities==1) | (df.Responsability==1) | (df.Understanding==1) | (df.BondingNeighbours==1))
df.loc[mask_index, ['PositiveImpact', 'Adaptarion', 'Bonding', 'TastesAndAbilities', 'Responsability', 'Understanding', 'BondingNeighbours']]

,PositiveImpact,Adaptarion,Bonding,TastesAndAbilities,Responsability,Understanding,BondingNeighbours
622,NaN,1,1,0,1,0,1
2207,NaN,1,1,1,0,0,0
4067,NaN,0,1,0,0,0,0
6549,NaN,0,1,1,0,1,0
6551,NaN,0,1,1,0,1,0
7281,NaN,0,1,0,0,0,0


In [1074]:
df.loc[mask_index, ['PositiveImpact']] = df.PositiveImpact.mode()[0]

---

### nan values of '**InterruptChildren**' and '**InterruptChildrenFrequency**' columns
The pattern of the next questions is "If yes, then answere....", so 

To start missing values of 'InterruptChildrenFrequency' are set to 0, then
- if **InterruptChildrenFrequency** values **>** 0 (**nan**)  then  '**InterruptChildren**' is set to 1 ("**yes**")  
- else drop istance

In [1075]:
# Assign 1  where 'InterruptChildren' is NaN and 'InterruptChildrenFrequency' si not
df.loc[ (df.InterruptChildrenFrequency.isna() == False) & df.InterruptChildren.isna(), ['InterruptChildren']] = 1

In [1076]:
# Drop if 'InterruptChildren' and 'InterruptChildrenFrequency' are NaN
df = df.loc[ (df.InterruptChildrenFrequency.isna() == False) | (df.InterruptChildren.isna() == False), :]

#### Note:
Most of missing values are due to the fact that the respondents replied 'False' to the question 'Interrupted working day to take care of children?'

In [1077]:
na_val_count = df.InterruptChildrenFrequency.isna().sum()
print(f"There are {na_val_count} missing values in 'InterruptChildrenFrequency' column")

There are 1547 missing values in 'InterruptChildrenFrequency' column


In [1078]:
# Assign 0 where 'InterruptChildrenFrequency' is NaN
df.loc[df.InterruptChildrenFrequency.isna(), 'InterruptChildrenFrequency'] = 0

#### TODO

In [1079]:
# ?????????????????????????????????? THIS ISN'T POSSIBLE
df.loc[ (df.InterruptChildren == 0) & (df.InterruptChildrenFrequency > 0), 'InterruptChildrenFrequency'].value_counts()

InterruptChildrenFrequency
1.0    79
3.0    22
2.0    22
Name: count, dtype: int64

---

### nan values of '**NegativeImpact**' column
The pattern of the next questions is "If yes, then answere...", so 
- if at least one of the next questions values is 1 (**yes**)  then  '**NegativeImpact**' is set to 2 ("**yes**")

In [1080]:
df.NegativeImpact.isna().sum()

np.int64(20)

In [1081]:
mask_index = df.NegativeImpact.isna() & ((df.Anxiety==1) | (df.TechnologyUse==1) | (df.Sadness==1) | (df.Arguments==1) | (df.FinancialLoss==1) | (df.Loneliness==1) | (df.LossFamilyMember==1) | (df.NegativeImpactOther==1) | (df.AlcoholMedicationDrug==1))

print(f"There are {(df.loc[mask_index]).shape[0]} NaN values in 'NegativeImpact' column \nThis is the head:")

df.loc[mask_index, 'NegativeImpact' : 'NegativeImpactOther'].head()

There are 16 NaN values in 'NegativeImpact' column 
This is the head:


,NegativeImpact,Anxiety,Loneliness,Sadness,LossFamilyMember,FinancialLoss,Arguments,AlcoholMedicationDrug,TechnologyUse,NegativeImpactOther
407,NaN,1,1,1,0,0,0,0,0,0
1648,NaN,1,0,0,0,0,1,0,1,0
1853,NaN,1,1,1,0,0,0,0,1,0
2311,NaN,0,0,0,0,0,0,0,1,0
3159,NaN,1,0,1,1,0,0,0,1,0


In [1082]:
df.loc[mask_index, 'NegativeImpact'] = df.NegativeImpact.mode()[0]

Drop all the remaining instances with missing value

In [1083]:
df.loc[df.NegativeImpact.isna(), 'NegativeImpact']

3483   NaN
4232   NaN
4490   NaN
4491   NaN
Name: NegativeImpact, dtype: float64

In [1084]:
df = df.loc[df.NegativeImpact.isna() == False, :]

---

### nan values of '**FamiliesCollaboration**' column
- Replace all the missing values with meddle values such as "**I don't know**"

In [1085]:
na_val_count = df.FamiliesCollaboration.isna().sum()
print(f"There are {na_val_count} NaN values")

There are 0 NaN values


In [1086]:
df.loc[df.FamiliesCollaboration.isna(), 'FamiliesCollaboration'] = 2

---

### nan values of '**SchoolSpace**' column
- Replace all the missing values with meddle values such as "**Maybe**"

In [1087]:
na_val_count = df.SchoolSpace.isna().sum()
print(f"There are {na_val_count} NaN values")

There are 5 NaN values


In [1088]:
df.loc[df.SchoolSpace.isna(), 'SchoolSpace'] = 2

---

#### The filter of the na values

At the end I remove all the istances with at least one NaN value

In [1089]:
# Columns with a least one nan value
na_columns = df.columns[df.isna().any()]

na_columns = na_columns.drop("Income")

# rows with a least one N nan value
na_rows = df[na_columns].isna().sum(axis=1) >= 1

df_lessToManyNa = df.loc[na_rows, na_columns]
df_lessToManyNa


,Gender,Leisure,Sports,Friends,ExtendedFamily,Past,Future,Sex,Reconciling,TimeOff,ChildrenSchookwork,Bedtime,HelpOnline,SchoolCommunication,ParentsGroups,ParentsCommunicationReturn,ActivitiesOutside
1,2.0,3.0,1.0,1.0,4.0,3.0,3.0,3.0,2.0,1.0,NaN,0.0,3.0,4.0,4.0,2.0,2.0
3,2.0,4.0,3.0,2.0,2.0,4.0,1.0,NaN,2.0,0.0,2.0,1.0,3.0,2.0,3.0,2.0,3.0
139,1.0,2.0,2.0,2.0,4.0,4.0,3.0,3.0,1.0,0.0,NaN,1.0,3.0,1.0,3.0,1.0,2.0
142,2.0,1.0,1.0,4.0,4.0,3.0,4.0,3.0,2.0,0.0,NaN,1.0,1.0,2.0,2.0,1.0,3.0
220,2.0,1.0,1.0,2.0,1.0,4.0,4.0,2.0,2.0,0.0,2.0,1.0,2.0,2.0,3.0,2.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7189,2.0,2.0,2.0,2.0,3.0,4.0,3.0,3.0,2.0,0.0,2.0,0.0,NaN,2.0,1.0,2.0,3.0
7204,2.0,2.0,1.0,2.0,3.0,4.0,3.0,3.0,2.0,0.0,NaN,1.0,4.0,1.0,1.0,2.0,1.0
7210,2.0,3.0,4.0,NaN,4.0,3.0,1.0,3.0,2.0,1.0,3.0,0.0,4.0,2.0,3.0,1.0,3.0
7262,2.0,1.0,1.0,2.0,2.0,3.0,3.0,2.0,2.0,0.0,3.0,1.0,NaN,1.0,1.0,3.0,2.0


In [1090]:
index = [i for i in np.array(df.index) if i not in df_lessToManyNa.index]

df = df.loc[index, :]

In [1095]:
df.shape

(6016, 109)

## FILE SAVING

In [1110]:
df.to_csv(path_or_buf="./Array_saved/X_without_null.csv", index=False)